# Convex optimization using CVX

In [125]:
import cvxpy as cp
import numpy as np

# Decision variables
x[0] = $F_p$: amplitude of powertrain force (control input) \
x[1] = $\dot{x}$: amplitude of velocity \
x[2] = $a$: binary indicator for whether powertrain max speed is exceeded

In [126]:
Fp = cp.Variable(pos=True)
xdot = cp.Variable(pos=True)
a = cp.Variable(pos=True)#boolean=True)
x = cp.vstack([Fp,xdot,a])

# Objective
Start by trying to maximize mechanical power (not electrical yet) \
max $F_p  \dot{x} = \frac{1}{2} x^T P x $ where $ P = \begin{bmatrix} 0 & 1 & 0 \\ 1 &  0 & 0 \\ 0 & 0 & 0 \end{bmatrix} $ \
This is a quadratic program and it would be ideal to leverage its structure.
P has eigenvalues [1, -1, 0] so it is not positive or negative definite and therefore the QP is not convex or concave. Still, since the objective is a posynomial, convexity/concavity are possible in log-log transformation, so we pursue geometric programming and examine the log-log curvature.

In [127]:
P = np.zeros((3,3))
P[0,1] = 1
P[1,0] = 1
objective = x[0] @ x[1] #cp.quad_form(x, P)
objective_2 = cp.inv_prod(x[0:2])
print('Objective curvature: ',objective.curvature)
print('Objective curvature: ',objective_2.curvature)

Objective curvature:  LOG-LOG CONVEX
Objective curvature:  CONVEX


This is a problem: in convex optimization it is allowed to minimize a convex function or maximize a concave function, but not to maximize a convex function

# Constraints
Let's try to implement the disjunctive constraint $F_p = 0$ if $\dot{x} \geq x_{max}$, and $F_p \leq F_{max}$ otherwise. This is more realistic to generator capabilities than simply constraining the speed to be within some threshold. (This is motivated separately from convexity, I can use this same disjunctive constraint even if the problem isn't convex). With this constraint I expect a solution of $\dot{x} \approx \dot{x}_{max}$, $F_p = F_{max}$, and $a=0$.
\
\
This can be implemented with four linear constraints using a "big M" formulation and the discrete variable $a \in \{0,1\}$ to control the disjunction: \
$ M a \geq \dot{x} - \dot{x}_{max} $ \
$ M (1 -a) \geq \dot{x}_{max} - \dot{x} $ \
$ F_p \leq F_{max} + a $ \
$ F_p \leq M_2 (1 - a) $

In [128]:
xdotmax = 3
Fmax = 5
M = 1e3
M2 = Fmax + 1

A = np.array([[0, 1, -M], [0, -1, M], [1, 0, -1], [1, 0, M2]])
b = np.array([[xdotmax], [-xdotmax + M], [Fmax], [M2]])

#constraints = [A @ x <= b]
constraints = [Fp <= Fmax, xdot <= xdotmax, a <= 0]
#print([constraint for constraint in constraints])
is_gdp = [constraint.is_dgp() for constraint in constraints]
print('constraint is GDP: ',is_gdp)
print('Ax-b curvature: ',(A@x-b).curvature)


constraint is GDP:  [True, True, False]
Ax-b curvature:  AFFINE


It is also a problem that the constraint does not have known curvature, which is required for convexity via geometric programming.
# Solver
Despite the known problems with the objective and constraints, we try solving the problem anyway to see what happens. We get an error as expected.

In [129]:
problem = cp.Problem(cp.Minimize(objective_2), constraints)
print('problem iS GDP: ',problem.is_dgp())
print('problem is DCP: ',problem.is_dcp())
print(cp.installed_solvers())
problem.solve(verbose=True)#gp=True)

print("Optimal value: ", problem.value)
print("Fp: ", x[0].value)
print("xdot: ", x[1].value)
print("a: ", x[2].value)

problem iS GDP:  False
problem is DCP:  True
(CVXPY) Nov 26 10:27:14 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')
['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'SCIP', 'SCIPY', 'SCS']
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Nov 26 10:27:14 PM: Your problem has 3 variables, 3 constraints, and 0 parameters.
(CVXPY) Nov 26 10:27:14 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 26 10:27:14 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 26 10:27:14 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 26 10:27:14 PM: Your problem is compiled with the CPP canonicalization backend.
----

# Conclusion
This has shown that the problem can't be solved as a convex quadratic program or geometric program as-is. Possible next steps:
1) Find another way to make the problem convex. For example, Zhong and Yeung 2018 (OMAE) shows that constraining the slew rate of PTO force can guarantee convexity for a single WEC, and Zhong and Yeung 2022 (Ocean Engineering) shows that constraining reactive power to zero (pure damping control) can guarantee convexity for an array.
2) Give up on convexity and try to solve it as a nonconvex QP with a different solver, which should still be faster than solving it as a NLP but would not have the benefits of convexity such as "free" sensitivities or global optimiality.
3) Give up on trying to leverage the QP structure, and use the current NLP solvers like SLSQP and interior point.